# JustWatch와 IMDb 간단한 데이터 전처리

In [581]:
# import
import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
import time
from tqdm import tqdm_notebook

from bs4 import BeautifulSoup
from selenium.common.exceptions import NoSuchElementException
import requests
import urllib.parse

In [582]:
# 엑셀 파일을 읽어옵니다.
df_us = pd.read_excel('../data/justwatch_us_age.xlsx', index_col=0)
df_us.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,add_original_title,add_english_title,justwatch_us_age
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021,https://www.imdb.com/title/tt10919420/,오리지날 타이틀 없음,Squid Game,TV-MA
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Mini Series 2022,https://www.imdb.com/title/tt13589004/,오리지날 타이틀 없음,The Twelve,TV-MA
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020,https://www.imdb.com/title/tt12079212/,誰是被害者,The Victims' Game,TV-MA
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021,https://www.imdb.com/title/tt12809988/,오리지날 타이틀 없음,Sweet Tooth,TV-14
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020,https://www.imdb.com/title/tt11612120/,오리지날 타이틀 없음,Sweet Home,TV-MA


In [583]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1089
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               1090 non-null   object
 1   original_title      855 non-null    object
 2   year                1090 non-null   int64 
 3   season_episode      1090 non-null   int64 
 4   runtime             1090 non-null   object
 5   genre               1090 non-null   object
 6   age_rating          803 non-null    object
 7   Production country  1089 non-null   object
 8   IMDb_title          1079 non-null   object
 9   IMDb_URL            1079 non-null   object
 10  add_original_title  1090 non-null   object
 11  add_english_title   1090 non-null   object
 12  justwatch_us_age    1090 non-null   object
dtypes: int64(2), object(11)
memory usage: 119.2+ KB


In [584]:
df_us.drop(['year', 'season_episode','runtime', 'genre', 'age_rating','Production country', 'IMDb_title','IMDb_URL'], axis=1, inplace=True)

In [585]:
# 엑셀 파일을 읽어옵니다.
df = pd.read_excel('../data/imdb_final.xlsx', index_col=0)
df.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_11,season_12,season_13,season_14,season_15,season_16,season_17,director,writer,actor
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak..."
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm..."
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si..."
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani..."
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ..."


In [586]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1089
Data columns (total 39 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1090 non-null   object 
 1   original_title      855 non-null    object 
 2   year                1090 non-null   int64  
 3   season_episode      1090 non-null   int64  
 4   runtime             1090 non-null   object 
 5   genre               1090 non-null   object 
 6   age_rating          803 non-null    object 
 7   Production country  1089 non-null   object 
 8   IMDb_title          1087 non-null   object 
 9   IMDb_URL            1087 non-null   object 
 10  total_rate          1083 non-null   float64
 11  total_count         1083 non-null   object 
 12  watchlist           841 non-null    object 
 13  popularity          503 non-null    object 
 14  user_review         849 non-null    object 
 15  critic_review       809 non-null    float64
 16  age_miss   

In [587]:
df = pd.merge(df, df_us, on=['title','original_title'], how='left')
df.info()							    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1092 non-null   object 
 1   original_title      857 non-null    object 
 2   year                1092 non-null   int64  
 3   season_episode      1092 non-null   int64  
 4   runtime             1092 non-null   object 
 5   genre               1092 non-null   object 
 6   age_rating          805 non-null    object 
 7   Production country  1091 non-null   object 
 8   IMDb_title          1089 non-null   object 
 9   IMDb_URL            1089 non-null   object 
 10  total_rate          1085 non-null   float64
 11  total_count         1085 non-null   object 
 12  watchlist           843 non-null    object 
 13  popularity          505 non-null    object 
 14  user_review         851 non-null    object 
 15  critic_review       811 non-null    float64
 16  age_mi

In [588]:
# 애니메이션, TV쇼 개별 삭제
df.drop(124, axis=0, inplace=True)
df.drop(151, axis=0, inplace=True)
df.drop(696, axis=0, inplace=True)
df.drop(826, axis=0, inplace=True)
df.drop(865, axis=0, inplace=True)
df.drop(873, axis=0, inplace=True)
df.drop(887, axis=0, inplace=True)
df.drop(913, axis=0, inplace=True)
df.drop(947, axis=0, inplace=True)
df.drop(1032, axis=0, inplace=True)
df.drop(1058, axis=0, inplace=True)

In [589]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1081 entries, 0 to 1091
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               1081 non-null   object 
 1   original_title      849 non-null    object 
 2   year                1081 non-null   int64  
 3   season_episode      1081 non-null   int64  
 4   runtime             1081 non-null   object 
 5   genre               1081 non-null   object 
 6   age_rating          798 non-null    object 
 7   Production country  1080 non-null   object 
 8   IMDb_title          1078 non-null   object 
 9   IMDb_URL            1078 non-null   object 
 10  total_rate          1074 non-null   float64
 11  total_count         1074 non-null   object 
 12  watchlist           838 non-null    object 
 13  popularity          504 non-null    object 
 14  user_review         845 non-null    object 
 15  critic_review       805 non-null    float64
 16  age_miss   

In [590]:
# '평점 없음'인 행들을 선택
rows_to_drop = df[df['season_1'] == '평점 없음'].index

# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)

# 시즌1 '평점 없음' 행 삭제 갯수
print(len(rows_to_drop))

153


In [591]:
# '2024년 공개작 포함'인 행들을 선택
rows_to_drop = df[df['season_1'] == '2024년 공개작 포함'].index
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)
# 시즌1 '2024년 공개작 포함' 행 삭제 갯수
len(rows_to_drop)

11

In [592]:
# '에피소드 정보 없음'인 행들을 선택
rows_to_drop = df[df['season_1'] == '에피소드 정보 없음'].index
# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)
# 시즌1 '에피소드 정보 없음' 행 삭제 갯수
print(len(rows_to_drop))

9


In [593]:
# 'season_1' 컬럼의 값이 None인 행들을 선택
rows_to_drop = df[df['season_1'].isna()].index

# 삭제된 행의 수 출력
print(len(rows_to_drop))

# 행 삭제 (inplace 사용)
df.drop(rows_to_drop, inplace=True)

3


In [594]:
df[795:805]

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,...,season_14,season_15,season_16,season_17,director,writer,actor,add_original_title,add_english_title,justwatch_us_age
885,섀도우,Shadow,2019,1,45min,"액션, 범죄, 드라마, 스릴러",NaN,남아프리카공화국,Shadow (TV Mini Series 2019),https://www.imdb.com/title/tt9817236/,...,NaN,NaN,NaN,NaN,"['Fred Wolmarans', 'Gareth Crocker']",['Gareth Crocker'],"['Pallance Dladla', 'Khathu Ramabulana', 'Aman...",오리지날 타이틀 없음,Shadow,TV-MA
886,크리미널: 프랑스,Criminal: France,2019,1,26min,"드라마, 범죄, 스릴러",15,프랑스,Criminal: France (TV Mini Series 2019),https://www.imdb.com/title/tt10986052/,...,NaN,NaN,NaN,NaN,['Frédéric Mermoud'],"['Antonin Martin-Hilbert', 'Frédéric Mermoud',...","['Margot Bancilhon', 'Laurent Lucas', 'Stéphan...",오리지날 타이틀 없음,Criminal: France,TV-MA
891,피의 시인,Bard of Blood,2019,2,45min,"액션, 드라마, 스릴러",NaN,인도,Bard of Blood (TV Series 2019,https://www.imdb.com/title/tt8069036/,...,NaN,NaN,NaN,NaN,['Ribhu Dasgupta'],"['Bilal Siddiqi', 'Mayank Tewari', 'Vidit Trip...","['Reapstar', 'Gurii', 'Abhi Brar', 'Emraan Has...",오리지날 타이틀 없음,Bard of Blood,TV-MA
892,스트레인지 스토리,పిట్టా కథలు,2021,1,37min,"드라마, 로맨스",15,인도,Pitta Kathalu (TV Series 2021),https://www.imdb.com/title/tt13878800/,...,NaN,NaN,NaN,NaN,"['B.V. Nandini Reddy', 'Nag Ashwin', 'Sankalp ...","['Lakshmi Bhupala', 'Nag Ashwin', 'Nanda Kisho...","['Shruti Haasan', 'Jagapathi Babu', 'Satyadev ...",పిట్టా కథలు,Pitta Kathalu,TV-MA
894,영혼 신부,彼岸之嫁,2020,1,51min,"스릴러, 드라마, 가족, 판타지",19,"말레이시아, 대만",The Ghost Bride (TV Mini Series 2020),https://www.imdb.com/title/tt9690302/,...,NaN,NaN,NaN,NaN,"['Shio Chuan Quek', 'Yuhang Ho']","['Anwari Ashraf', 'Dan Hamamura', 'June Tan', ...","['Peijia Huang', 'Kang Ren Wu', 'Ludi Lin', 'K...",彼岸之嫁,The Ghost Bride,TV-14
896,멈출 수 없어,Desenfrenadas,2020,1,34min,드라마,19,멕시코,Unstoppable (TV Series 2020),https://www.imdb.com/title/tt11120674/,...,NaN,NaN,NaN,NaN,"['Diego Martínez Ulanosky', 'Elisa Miller', 'J...","['Ashauri López', 'Diego Martínez Ulanosky', '...","['Tessa Ia', 'Bárbara López', 'Lucía Uribe', '...",Desenfrenadas,Unstoppable,TV-MA
898,새비지 리듬,Ritmo salvaje,2022,1,38min,"드라마, 음악",NaN,콜롬비아,Savage Rhythm (TV Series 2022),https://www.imdb.com/title/tt14494806/,...,NaN,NaN,NaN,NaN,"['Andres Beltran', 'Rafael Martínez Moreno', '...","['Diego Vivanco', 'Juliana Barrera', 'Valeria ...","['Paulina Dávila', 'Brooke Newton', 'Angela Ca...",Ritmo salvaje,Savage Rhythm,TV-MA
899,겟 이븐: 우리는 치어 스쿼드,Rebel Cheer Squad: A Get Even Series,2022,1,29min,"드라마, 가족, 스릴러",12,영국,Rebel Cheer Squad: A Get Even Series,https://www.imdb.com/title/tt18268134/,...,NaN,NaN,NaN,NaN,"['Claire Tailyour', 'Nigel Douglas']","['Dan Braham', 'Gretchen McNeil', 'Holly Phill...","['Lashay Anderson', 'Amelia Brooks', ""Ashling ...",오리지날 타이틀 없음,Rebel Cheer Squad: A Get Even Series,TV-PG
902,3일의 크리스마스,Three Days of Christmas,2019,1,58min,드라마,15,스페인,Three Days of Christmas (TV Mini Series 2019),https://www.imdb.com/title/tt9731254/,...,NaN,NaN,NaN,NaN,['Pau Freixas'],"['Clara Esparrach', 'Ivan Mercadé', 'Marta Arm...","['Victoria Abril', 'Carla Tous', 'Jay Britton'...",오리지날 타이틀 없음,Three Days of Christmas,연령 정보 없음
904,루나 파크,Luna Park,2021,1,50min,"드라마, 로맨스",19,이탈리아,Luna Park (TV Series 2021),https://www.imdb.com/title/tt15349110/,...,NaN,NaN,NaN,NaN,"['Anna Negri', ""Leonardo D'Agostini""]",['Isabella Aguilar'],"['Lia Grieco', 'Guglielmo Poggi', 'Simona Taba...",오리지날 타이틀 없음,Luna Park,TV-MA


In [595]:
# 컬럼명 rename
df.rename(columns={'Production country': 'production_country','season_episode':'total_season_num','IMDb_title':'imdb_title','IMDb_URL':'imdb_url', 'add_english_title':'english_title'}, inplace=True)

In [596]:
# production_country 결측치 채우고 영어 이름(china) 한국이름으로 바꾸기
df['production_country'] = df['production_country'].fillna('China')

In [597]:
# 번역할 나라 이름 딕셔너리
countries_dict = {'China': '중국', 'Arjantin': '아르헨티나', '필리핀 제도': '필리핀'}

# 각 행의 값을 번역하는 함수
def translate_country(country):
    countries = country.split(', ')
    translated_countries = (countries_dict.get(c, c) for c in countries)
    return ', '.join(translated_countries)

# 'country' 열에 번역 함수 적용
df['production_country'] = df['production_country'].apply(translate_country)

In [598]:
df['production_country'].unique()

array(['대한민국', '벨기에', '대만', '미국', '덴마크', '독일', '일본', '태국', '캐나다', '그리스',
       '영국', '스페인', '미국, 영국', '아이슬란드, 덴마크, 핀란드, 스웨덴, 노르웨이, 독일, 프랑스, 영국',
       '미국, 프랑스', '중국', '호주', '스웨덴', '이탈리아', '케냐', '인도', '호주, 미국', '스위스',
       '필리핀', '영국, 미국', '남아프리카공화국', '폴란드', '브라질', '미국, 캐나다', '콜롬비아', '터키',
       '룩셈부르크', '핀란드', '이스라엘', '프랑스', '멕시코', '벨기에, 네덜란드', '덴마크, 노르웨이',
       '노르웨이', '러시아', '미국, 일본', '영국, 프랑스', '요르단', '프랑스, 미국', '캐나다, 영국',
       '인도네시아', '영국, 인도', '아르헨티나', '아이슬란드', '오스트리아', '포르투갈',
       '프랑스, 독일, 스페인, 영국', '아일랜드, 영국', '캐나다, 프랑스, 독일', '이스라엘, 노르웨이',
       '대한민국, 미국', '미국, 대한민국', '네덜란드', '멕시코, 스페인', '칠레', '뉴질랜드, 호주',
       '사우디아라비아', '나이지리아', '이집트', '일본, 미국', '미국, 멕시코', '쿠웨이트',
       '나이지리아, 미국', '말레이시아, 대만', '사우디아라비아, 인도', '벨기에, 룩셈부르크',
       '사우디아라비아, 튀니지, 이집트, 모로코, 레바논', '독일, 오스트리아', '체코 공화국, 독일',
       '영국, 스페인', '스페인, 브라질', '레바논'], dtype=object)

In [599]:
df.head()

,title,original_title,year,total_season_num,runtime,genre,age_rating,production_country,imdb_title,imdb_url,...,season_14,season_15,season_16,season_17,director,writer,actor,add_original_title,english_title,justwatch_us_age
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak...",오리지날 타이틀 없음,Squid Game,TV-MA
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm...",오리지날 타이틀 없음,The Twelve,TV-MA
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si...",誰是被害者,The Victims' Game,TV-MA
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani...",오리지날 타이틀 없음,Sweet Tooth,TV-14
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ...",오리지날 타이틀 없음,Sweet Home,TV-MA


In [600]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 0 to 1091
Data columns (total 42 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               905 non-null    object 
 1   original_title      724 non-null    object 
 2   year                905 non-null    int64  
 3   total_season_num    905 non-null    int64  
 4   runtime             905 non-null    object 
 5   genre               905 non-null    object 
 6   age_rating          650 non-null    object 
 7   production_country  905 non-null    object 
 8   imdb_title          905 non-null    object 
 9   imdb_url            905 non-null    object 
 10  total_rate          904 non-null    float64
 11  total_count         904 non-null    object 
 12  watchlist           815 non-null    object 
 13  popularity          501 non-null    object 
 14  user_review         808 non-null    object 
 15  critic_review       778 non-null    float64
 16  age_miss    

In [601]:
# 'min'을 '분'으로 바꾸는 함수
def replaceMin(string):
    return string.replace('min', '분')

# 시간을 분으로 변환하는 함수
def getTime(runtime):
    # '시간'과 '분'을 기준으로 분리
    hours, minutes = 0, 0
    if '시간' in runtime:
        hours = int(runtime.split('시간')[0].strip())
    if '분' in runtime:
        if '시간' in runtime:
            minutes = int(runtime.split('시간')[1].split('분')[0].strip())
        else:
            minutes = int(runtime.split('분')[0].strip())
    # 시간과 분을 합쳐서 분으로 계산
    total_minutes = hours * 60 + minutes
    return total_minutes

# 'min'을 '분'으로 변경
df['runtime'] = df['runtime'].apply(replaceMin)
# 'runtime' 컬럼의 값을 변환
df['runtime'] = df['runtime'].apply(getTime)

In [602]:
# 수식어에 대한 값을 변환하는 함수
def convertValue(value):
    try:
        # ',' 제거
        value = value.replace(',', '')
        # 숫자와 수식어 분리
        if 'K' in value:
            multiplier = 1000
            value = float(value.replace('K', ''))
        elif 'M' in value:
            multiplier = 1000000
            value = float(value.replace('M', ''))
        elif '만' in value:
            multiplier = 10000
            value = float(value.replace('만', ''))
        elif '천' in value:
            multiplier = 1000
            value = float(value.replace('천', ''))
        else:
            multiplier = 1
            value = float(value)
        # 값에 수식어를 곱하고 정수로 변환하여 반환
        return int(value * multiplier)
    except AttributeError:
        # 숫자가 아닌 값이 들어올 경우 그대로 반환
        return value

# 해당 컬럼들의 값을 변환
df['total_count'] = df['total_count'].apply(convertValue)
df['watchlist'] = df['watchlist'].apply(convertValue)
df['popularity'] = df['popularity'].apply(convertValue)
df['user_review'] = df['user_review'].apply(convertValue)
df['critic_review'] = df['critic_review'].apply(convertValue)

In [603]:
df.head()

,title,original_title,year,total_season_num,runtime,genre,age_rating,production_country,imdb_title,imdb_url,...,season_14,season_15,season_16,season_17,director,writer,actor,add_original_title,english_title,justwatch_us_age
0,Ojingeo Geim,Squid Game,2021,2,55,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,...,NaN,NaN,NaN,NaN,['Hwang Dong-hyuk'],['Hwang Dong-hyuk'],"['Lee Jung-jae', 'Park Hae-soo', 'Yasushi Iwak...",오리지날 타이틀 없음,Squid Game,TV-MA
1,12인의 심판자,The Twelve,2019,2,54,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,...,NaN,NaN,NaN,NaN,"['Kaat Beels', 'Wouter Bouvijn']","['Bert Van Dael', 'Nele Meirhaeghe', 'Roel Mon...","['Luc De Ruelle', 'Maaike Neuville', 'Tom Verm...",오리지날 타이틀 없음,The Twelve,TV-MA
2,희생자 게임,誰是被害者,2020,2,60,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,...,NaN,NaN,NaN,NaN,"['David Chuang', 'Kuan-Chung Chen']","['Joyce Liu', 'Jui-Liang Hsu', 'Shih-Keng Chie...","['Wei-Ning Hsu', 'Hsiao-chuan Chang', 'Shih-Si...",誰是被害者,The Victims' Game,TV-MA
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,...,NaN,NaN,NaN,NaN,"['Alexis Ostrander', 'Carol Banker', 'Ciarán F...","['Beth Schwartz', 'Bo Yeon Kim', 'Carly Woodwo...","['Nonso Anozie', 'Christian Convery', 'Stefani...",오리지날 타이틀 없음,Sweet Tooth,TV-14
4,Sweet Home,NaN,2020,3,60,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,...,NaN,NaN,NaN,NaN,"['Jang Young-woo', 'Lee Eung-bok', 'Park So-hy...","['Hong So-ri', 'Hwang Young-Chan', 'Kim Hyung-...","['Song Kang', 'Lee Jin-wook', 'Lee Si-young', ...",오리지날 타이틀 없음,Sweet Home,TV-MA


In [604]:
# 리스트가 아닌 문자열로 저장됨
print(len(df['production'][1]))
print(df['production'][1][0])

64
[


In [605]:
# 문자열을 리스트로 변환하는 함수
def parseStringToList(string):
    if pd.isna(string):  # NaN 값인 경우 빈 리스트 반환
        return []
    # 문자열에서 각 요소를 구분하는 쉼표로 분할
    elements = string.split(', ')
    # 각 요소에서 양쪽의 작은따옴표 제거
    elements = [element.strip("['").strip("']") for element in elements]
    return elements

# 'production','distributor' 컬럼의 값을 변환
df['production'] = df['production'].apply(parseStringToList)
df['distributor'] = df['distributor'].apply(parseStringToList)

In [606]:
# 리스트가 아닌 문자열로 저장됨
print(len(df['production'][1]))
print(df['production'][1][0])

2
Vlaamse Radio en Televisie (VRT)


In [607]:
# 'imdb_season_num' 컬럼 생성 및 초기화
df['imdb_season_num'] = np.nan

# 각 season 컬럼에 대해 순회하면서 해당 시즌이 없거나 '평점없음'인 경우에 해당하는 값으로 설정
for i in range(2, 18):
    season_col = f'season_{i}'
    condition = (df[season_col].isnull()) | (df[season_col] == '평점 없음')
    df.loc[condition & df['imdb_season_num'].isnull(), 'imdb_season_num'] = i - 1

# season_17에 데이터가 있는 경우 imdb_season_num을 17로 설정
if 'season_17' in df.columns and df['season_17'].notnull().any():
    df.loc[df['season_17'].notnull(), 'imdb_season_num'] = 17

# 결과 출력
print(df[['title', 'imdb_season_num']])


                        title  imdb_season_num
0                Ojingeo Geim              1.0
1                    12인의 심판자              2.0
2                      희생자 게임              1.0
3          스위트 투스: 사슴뿔을 가진 소년              2.0
4                  Sweet Home              2.0
...                       ...              ...
1081                      레일라              1.0
1084              100만 엔의 여인들              1.0
1085                  킬러 코미디언              1.0
1087                       달러              1.0
1091  엘리트들, 못다 한 이야기: 나디아 구스만              1.0

[905 rows x 2 columns]


In [608]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 905 entries, 0 to 1091
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               905 non-null    object 
 1   original_title      724 non-null    object 
 2   year                905 non-null    int64  
 3   total_season_num    905 non-null    int64  
 4   runtime             905 non-null    int64  
 5   genre               905 non-null    object 
 6   age_rating          650 non-null    object 
 7   production_country  905 non-null    object 
 8   imdb_title          905 non-null    object 
 9   imdb_url            905 non-null    object 
 10  total_rate          904 non-null    float64
 11  total_count         904 non-null    float64
 12  watchlist           815 non-null    float64
 13  popularity          501 non-null    float64
 14  user_review         808 non-null    float64
 15  critic_review       778 non-null    float64
 16  age_miss    

In [609]:
df.drop(['total_rate', 'total_count', 'watchlist','popularity', 'user_review', 'critic_review'], axis=1, inplace=True)

In [610]:
# 엑셀 파일을 읽어옵니다.
df_new = pd.read_excel('../data/IMDb_first_page_결측치_수정본_콤마제거.xlsx', index_col=0)
df_new.head()

,title,original_title,year,season_episode,runtime,genre,age_rating,Production country,IMDb_title,IMDb_URL,total_rate,total_count,watchlist,popularity,user_review,critic_review,age_miss
0,Ojingeo Geim,Squid Game,2021,2,55min,"액션, 드라마, 스릴러",19,대한민국,Squid Game (TV Series 2021),https://www.imdb.com/title/tt10919420/,8.0,549000.0,250000,237,2600,109,18
1,12인의 심판자,The Twelve,2019,2,54min,"드라마, 스릴러",NaN,벨기에,The Twelve (TV Series 2019–2023),https://www.imdb.com/title/tt7605396/,7.4,6500.0,9800,2530,89,7,18
2,희생자 게임,誰是被害者,2020,2,1시간 0분,"드라마, 범죄, 스릴러",19,대만,The Victims' Game (TV Series 2020),https://www.imdb.com/title/tt12079212/,7.3,1900.0,2100,0,25,0,18
3,스위트 투스: 사슴뿔을 가진 소년,Sweet Tooth,2021,3,52min,"SF, 드라마, 판타지, 액션",15,미국,Sweet Tooth (TV Series 2021),https://www.imdb.com/title/tt12809988/,7.7,68000.0,97000,41,791,86,15
4,Sweet Home,NaN,2020,3,1시간 0분,"스릴러, 드라마, 판타지, 공포, SF, 범죄",NaN,대한민국,Sweet Home (TV Series 2020),https://www.imdb.com/title/tt11612120/,7.3,33000.0,43000,940,516,37,18


In [611]:
df_new.drop(['original_title', 'year', 'season_episode', 'runtime', 'genre', 'age_rating','Production country', 'IMDb_title','age_miss'], axis=1, inplace=True)

In [612]:
# 컬럼명 rename
df_new.rename(columns={'IMDb_URL':'imdb_url'}, inplace=True)

In [613]:
df = pd.merge(df, df_new, on=['title','imdb_url'], how='left')
df.info()	

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 43 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               905 non-null    object 
 1   original_title      724 non-null    object 
 2   year                905 non-null    int64  
 3   total_season_num    905 non-null    int64  
 4   runtime             905 non-null    int64  
 5   genre               905 non-null    object 
 6   age_rating          650 non-null    object 
 7   production_country  905 non-null    object 
 8   imdb_title          905 non-null    object 
 9   imdb_url            905 non-null    object 
 10  age_miss            861 non-null    object 
 11  production          905 non-null    object 
 12  distributor         905 non-null    object 
 13  season_1            905 non-null    object 
 14  season_2            295 non-null    object 
 15  season_3            146 non-null    object 
 16  season_4

In [614]:
# 인덱스 변경
df.reset_index(drop=True,inplace=True)# 새로운 인덱스 부여

In [615]:
# 새로운 컬럼 순서 지정
columns = [
    'title', 'english_title', 'year', 'imdb_season_num', 'runtime', 'genre',
    'age_rating', 'age_miss','justwatch_us_age', 'production_country',
    'total_count', 'total_rate', 'watchlist', 'popularity', 'user_review', 'critic_review',
    'production', 'distributor', 'season_1', 'season_2', 'season_3', 'season_4',
    'season_5', 'season_6', 'season_7', 'season_8', 'season_9', 'season_10',
    'season_11', 'season_12', 'season_13', 'season_14', 'season_15', 'season_16',
    'season_17', 'director', 'writer', 'actor',
    'imdb_title', 'imdb_url'
]
# 데이터프레임의 컬럼 순서 변경
df = df[columns]

In [616]:
# 컬럼명 rename
df_new.rename(columns={'imdb_season_num':'total_season_num'}, inplace=True)

In [617]:
# english_title 결측치 채우기
df['english_title'] = df['english_title'].fillna('1899')

In [618]:
df_us.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1090 entries, 0 to 1089
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               1090 non-null   object
 1   original_title      855 non-null    object
 2   add_original_title  1090 non-null   object
 3   add_english_title   1090 non-null   object
 4   justwatch_us_age    1090 non-null   object
dtypes: object(5)
memory usage: 51.1+ KB


In [619]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 905 entries, 0 to 904
Data columns (total 40 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   title               905 non-null    object 
 1   english_title       905 non-null    object 
 2   year                905 non-null    int64  
 3   imdb_season_num     905 non-null    float64
 4   runtime             905 non-null    int64  
 5   genre               905 non-null    object 
 6   age_rating          650 non-null    object 
 7   age_miss            861 non-null    object 
 8   justwatch_us_age    904 non-null    object 
 9   production_country  905 non-null    object 
 10  total_count         901 non-null    float64
 11  total_rate          901 non-null    float64
 12  watchlist           901 non-null    float64
 13  popularity          901 non-null    float64
 14  user_review         901 non-null    float64
 15  critic_review       901 non-null    float64
 16  producti

In [621]:
# 저장
df.to_csv('../data/imdb_final_3.csv', sep="/", encoding="utf-8")

df.to_excel('../data/imdb_final_3.xlsx')